# Bloomberg BQuant Spotlight Webinar Series: Balancing Act
This is a companion notebook to the "Understanding the Financial Statement Impact of Operating Leases" webinar.

In [ ]:
import bqplot as bqp
from bqplot.interacts import BrushSelector
import pandas as pd
import numpy as np
from ipywidgets import Dropdown, HBox, VBox, HTML
import bqwidgets as bqw
import bql
bq=bql.Service()

In [ ]:
params={'currency':'USD','fa_filing_status':'MRXP'}
qparams={'currency':'USD','fa_filing_status':'MRXP','fa_period_type':'Q'}

<h5 style='color:LIGHTSKYBLUE'>Proxy Metric - Operating Lease Percentage of Liabilities</h5>

In [ ]:
total_ol_liability=bq.func.znav(bq.data.bs_total_operating_lease_liabs(**params))
debt=bq.func.znav(bq.data.bs_tot_liab2(**params))

op_lease_pct_of_liabilities=bq.func.if_(bq.func.or_(bq.func.equals(total_ol_liability,0),bq.func.equals(debt,0)),
                                       0,
                                       bq.func.dropna(total_ol_liability/debt))*100

<h5 style='color:LightSeaGreen'>Pre-ASC842 EBITDA</h5>

In [ ]:
def bq_old_ebitda(params):
    grid=bq.data.eqy_fund_ind()
    da=bq.data.cf_depr_amort(**params)
    
    oi=bq.data.is_oper_inc(**params)
    reit_oi=bq.data.ebit(**params)
    
    industrial=oi+da
    financial=oi+da+bq.func.znav(bq.data.is_int_expenses(**params))
    utility=oi+da+bq.func.znav(bq.data.is_total_d_and_a_adjustment(**params))
    reit=reit_oi+da
    
    old_ebitda=bq.func.if_(bq.func.in_(grid,['Industrial']),industrial,
                           bq.func.if_(bq.func.in_(grid,['Financial']),financial,
                                       bq.func.if_(bq.func.in_(grid,['Utility']),utility,
                                                   bq.func.if_(bq.func.in_(grid,['REIT']),reit,bql.NA))))
    return old_ebitda
old_ebitda=bq_old_ebitda(params)
old_qebitda=bq_old_ebitda(qparams)

<h5 style='color:LightSeaGreen'>Pre-ASC842 Enterprise Value</h5>

In [ ]:
old_ev=bq.data.curr_entp_val(**params)-bq.func.znav(bq.data.bs_total_operating_lease_liabs(**params))

<h5 style='color:LightSeaGreen'>Pre-ASC842 Total Debt</h5>

In [ ]:
old_debt=bq.data.short_and_long_term_debt(**params)-bq.func.znav(bq.data.bs_total_operating_lease_liabs(**params))

<h5 style='color:LIGHTSKYBLUE'>Post-ASC842 vs. Pre-ASC842</h5>

In [ ]:
ev=bq.data.curr_entp_val(**params)
ebitda=bq.data.ebitda(**params)
qebitda=bq.data.ebitda(**qparams)
sales=bq.data.sales_rev_turn(**params)
debt=bq.data.short_and_long_term_debt(**params)

ev_ebitda=bq.func.if_(ebitda>0,ev/ebitda,bql.NA)
old_ev_ebitda=bq.func.if_(old_ebitda>0,old_ev/old_ebitda,bql.NA)

disc_ev_ebitda=bq.func.if_(ev_ebitda/old_ev_ebitda>0,
                          ev_ebitda/old_ev_ebitda-1,
                          bql.NA)*100

ebitda_margin=bq.func.if_(sales>0,ebitda/sales,bql.NA)
old_ebitda_margin=bq.func.if_(sales>0,old_ebitda/sales,bql.NA)

disc_ebitda_margin=bq.func.if_(ebitda_margin/old_ebitda_margin>0,
                               ebitda_margin/old_ebitda_margin-1,
                               bql.NA)*100

debt_ebitda=bq.func.if_(ebitda>0,debt/ebitda,bql.NA)
old_debt_ebitda=bq.func.if_(old_ebitda>0,old_debt/old_ebitda,bql.NA)

disc_debt_to_ebitda=(debt_ebitda/old_debt_ebitda-1)*100

disc_10q_ebitda=bq.func.if_(bq.func.and_(bq.func.equals(old_qebitda,0)==False,qebitda/old_qebitda>0),
                            qebitda/old_qebitda-1,
                            bql.NA)*100

<h5 style='color:ORANGE'>Data Request</h5>

In [ ]:
req_d={'Name':bq.data.name(),
        'Sector':bq.data.gics_sector_name(),
        'Industry':bq.data.gics_industry_name(),
        'FA Class':bq.data.eqy_fund_ind(),
        'Op Lease Pct of Liabilities':op_lease_pct_of_liabilities,
        'EV to EBITDA':disc_ev_ebitda,
        'EBITDA Margin':disc_ebitda_margin,
        'Debt to EBITDA':disc_debt_to_ebitda,
        'Last 10-Q EBITDA':disc_10q_ebitda}


univ_filter_criteria=bq.func.and_(bq.func.znav(bq.data.bs_total_operating_lease_liabs(**params))>0,
                                  bq.func.in_(bq.data.eqy_fund_ind(),['Industrial','REIT','Utility','Financial']))
univ=bq.univ.filter(bq.univ.members('SPX Index'),univ_filter_criteria)
req=bql.Request(univ,req_d)

In [ ]:
bqexec=bq.execute(req)

In [ ]:
reference_columns=['Name','Sector','Industry','FA Class','Op Lease Pct of Liabilities']
discrepancy_columns=['Last 10-Q EBITDA', 'EV to EBITDA','EBITDA Margin', 'Debt to EBITDA']

In [ ]:
df_cols=[]
for col in reference_columns+discrepancy_columns:
    df_cols.append(bqexec.get(col).df()[col])

data=pd.concat(df_cols,axis=1).reset_index().rename(columns={'ID':'Ticker'})

In [ ]:
data.head()

In [ ]:
data_clean=data.copy()
for col in discrepancy_columns:
    data_clean[col]=data_clean[col].clip(lower=data[col].quantile(0.05),upper=data[col].quantile(0.95))

In [ ]:
data_clean=data_clean.round(decimals=2)

<h5 style='color:ORANGE'>Post-ASC842 vs. Pre-ASC842 Visualization</h5>

In [ ]:
# Data source
# data_clean

data_cols=['Op Lease Pct of Liabilities']+discrepancy_columns

# Create scales
scale_x = bqp.LinearScale()
scale_y = bqp.LinearScale()

c_sc=bqp.OrdinalColorScale()
ttp_flds=['name','color']
ttp_lbls=['Name','Sector']
ttp=bqp.Tooltip(fields=ttp_flds,labels=ttp_lbls)

# Create marks
mark_scatter = bqp.Scatter(x=data_clean[data_cols[0]],
                           y=data_clean[data_cols[1]],
                           scales={'x': scale_x, 'y': scale_y,'color':c_sc},
                           default_size=48,
                           color=data_clean['Industry'],
                          names=data_clean['Name'],
                          display_names=False,
                          tooltip=ttp)

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label=data_cols[0])
axis_y = bqp.Axis(scale=scale_y,
                  orientation='vertical',
                  tick_format='0.0f',
                  label=data_cols[1])

# Create selector
selector = BrushSelector(x_scale=scale_x,
                         y_scale=scale_y,
                         marks=[mark_scatter])

# Create Figure
figure = bqp.Figure(marks=[mark_scatter],
                    axes=[axis_x, axis_y],
                    animation_duration=500,
                    layout={'width':'99%', 'height':'400px'},
                    padding_x=0.05,
                    title='S&P 500 ASC 842 Impact',
                    title_style={'font-size': '22px'},
                    padding_y=0.05,
                    interaction=selector,
                    fig_margin={'top': 50, 'bottom': 60,
                                'left': 50, 'right':30})


# Create dropown widgets
dropdown_x = Dropdown(description='X axis',
                      options=data_cols,
                      value=data_cols[0])
dropdown_y = Dropdown(description='Y axis',
                      options=data_cols,
                      value=data_cols[1])

# Define callback function for dropdown widgets
def update_plot(evt):
    if evt is not None:
        new_value = evt['new']
        if evt['owner'] == dropdown_x:
            mark_scatter.x = data_clean[new_value]
            axis_x.label = new_value
        elif evt['owner'] == dropdown_y:
            mark_scatter.y = data_clean[new_value]
            axis_y.label = new_value


# Define callback function for selections
def on_select(evt):
    if evt is not None and evt['new'] is not None:
        indices = evt['new']
        datagrid.data = data_clean.iloc[indices].reset_index()

# Bind callback to the dropdown widgets
dropdown_x.observe(update_plot, names=['value'])
dropdown_y.observe(update_plot, names=['value'])
mark_scatter.observe(on_select, names=['selected'])

# Create datagrid
col_defs=[{'children': [{'field': 'Ticker', 'headerName': 'Ticker', 'width': 170},
           {'field': 'Name', 'headerName': 'Name', 'width': 190},
           {'field': 'Sector', 'headerName': 'Sector', 'width': 190},
           {'field': 'Industry', 'headerName': 'Industry', 'width': 190},
           {'field': 'FA Class', 'headerName': 'FA Class', 'width': 96},
           {'field': 'Op Lease Pct of Liabilities',
            'headerName': 'Op Lease Pct of Liabilities',
            'width': 240}],
          'headerName': 'Company'},
         {'children': [{'field': 'Last 10-Q EBITDA',
            'headerName': 'Last 10-Q EBITDA',
            'width': 192},
           {'field': 'EV to EBITDA', 'headerName': 'EV to EBITDA', 'width': 144},
           {'field': 'EBITDA Margin', 'headerName': 'EBITDA Margin', 'width': 156},
           {'field': 'Debt to EBITDA', 'headerName': 'Debt to EBITDA', 'width': 168}],
          'headerName': '% Discrepancy in ASU 2016-02 Impacted Data'}]
datagrid = bqw.DataGrid(data=data_clean,column_defs=col_defs)

# Create Box containers
widget_box = HBox([dropdown_x, dropdown_y], layout={'margin': '10px'})
app_container = VBox([figure, widget_box, datagrid],
                     layout={'width':'100%'})

# Display the visualization
app_container
